In [1]:
import time
import datetime
import requests
import csv

In [2]:
#!pip install tqdm
from tqdm import notebook

In [3]:
session = requests.Session()
with open ("token.txt", "r") as f:
    x = f.readline()
session.headers.update({"Authorization": "token "+x.strip()})

In [4]:
error_url = []
    
def FetchDataFromGitHubRestApi(url):
    response = session.get(url)
    if response.status_code == 200:
        fetched_data = response.json()
        poll_interval = None
        poll_interval = response.headers.get('X-Poll-Interval')
        if poll_interval:
            time.sleep(int(poll_interval))
        return fetched_data
    else:
        error_url.append(url)

def Select(field, fetched_data):
    # Select Field From Fetched Data
    if fetched_data[field]:
        if type(fetched_data[field]) == str:
            try:
                output = str(fetched_data[field])
            except:
                output = fetched_data[field]
        else:
            output = str(fetched_data[field])
        output = output.replace("'",'').replace(", ",'/').replace(",",'/').replace('[','').replace(']','')
        if output == " " or output == "":
            output = "None"
    else:
        output = "None"
    return output

In [5]:
with open ("8K_github_username.txt", "r", encoding="utf8") as f:
    all_username = f.readlines()

In [6]:
all_username = [x.strip() for x in all_username]

In [7]:
len(all_username)

8932

In [8]:
first_half = all_username[:4999]
second_half = all_username[4999:]

In [9]:
print(len(first_half)+len(second_half))

8932


In [10]:
def CollectUserProfileData(writer, username, not_a_user_or_no_data, c):
    try:
        user_alias = "usr"+str(c)
        
        profile_data = FetchDataFromGitHubRestApi("https://api.github.com/users/"+username)
    
        type_ = Select("type", profile_data)
        location = Select("location", profile_data)
        hireable = Select("hireable", profile_data)
        public_repos = Select("public_repos", profile_data)
        public_gists = Select("public_gists", profile_data)
        followers = Select("followers", profile_data)
        following = Select("following", profile_data)
        created_at = Select("created_at", profile_data)
        updated_at = Select("updated_at", profile_data)
    
        if type_ == "User":
            writer.writerow([user_alias, type_, hireable, location, followers, following, public_repos, public_gists, created_at, updated_at])
            return True
        else:
            not_a_user_or_no_data.append(username)
            return False
    
    except:
        not_a_user_or_no_data.append(username)
        return False

In [11]:
print("now ",datetime.datetime.now().hour,":", datetime.datetime.now().minute)

now  20 : 1


In [12]:
not_a_user_or_no_data = []

with open('all_user_profile_data.csv', 'w', newline='', encoding='utf8') as f:
    
    writer = csv.writer(f)
    
    writer.writerow(["user_alias", "type", "hireable", "location", "followers", "following", "public_repos", "public_gists", "created_at", "updated_at"])

    c = 0
    for username in notebook.tqdm(first_half):
        count = CollectUserProfileData(writer, username, not_a_user_or_no_data, c)
        if count == True:
            c+=1

    print("first half complete!")

    print("wait for request limit")
    wait_time = 60 - datetime.datetime.now().minute
    print("wait_time = ", wait_time+1)
    for n in notebook.tqdm(range(wait_time+1)):
        time.sleep(60)
    print("request limit = 4999")
    
    for username in notebook.tqdm(second_half):
        count = CollectUserProfileData(writer, username, not_a_user_or_no_data, c)
        if count == True:
            c+=1

print("second half complete!")

  0%|          | 0/4999 [00:00<?, ?it/s]

first half complete!
wait for request limit
wait_time =  31


  0%|          | 0/31 [00:00<?, ?it/s]

request limit = 4999


  0%|          | 0/3933 [00:00<?, ?it/s]

second half complete!


In [13]:
lines = [x+"\n" for x in not_a_user_or_no_data]
with open('not_a_user_or_no_data.txt', 'w', newline='', encoding='utf8') as f:
    for line in lines:
        f.write(line)
print("done!")

done!


In [14]:
print(len(all_username), len(not_a_user_or_no_data))

8932 349


In [15]:
print(len(all_username) - len(not_a_user_or_no_data))

8583
